In [1]:
import numpy as np
from scipy.spatial.distance import cdist
import scprep
import torch
import sys
sys.path.append('..')
from utils import sinkhorn_knopp_unbalanced

In [2]:
def load_data_full(datafile):
    dat = np.load(datafile)
    return dat['pca'][:, :10], dat['sample_labels']

def get_transform_matrix(gamma, a, epsilon=1e-8):
    """Return matrix such that T @ a = b
    gamma : gamma @ 1 = a; gamma^T @ 1 = b
    """
    return (np.diag(1.0 / (a + epsilon)) @ gamma).T

def get_growth_coeffs(gamma, a, epsilon=1e-8, normalize=False):
    T = get_transform_matrix(gamma, a, epsilon)
    unnormalized_coeffs = np.sum(T, axis=0)
    if not normalize:
        return unnormalized_coeffs
    return unnormalized_coeffs / np.sum(unnormalized_coeffs) * len(unnormalized_coeffs)

In [3]:
def get_all_growth_coeffs(alpha):
    gcs = []
    for i in range(len(dfs) - 1):
        a, b = dfs[i], dfs[i + 1]
        m, n = a.shape[0], b.shape[0]
        M = cdist(a, b)
        entropy_reg = 0.1
        reg_1, reg_2 = alpha, 10000
        gamma = sinkhorn_knopp_unbalanced(
            np.ones(m) / m, np.ones(n) / n, M, entropy_reg, reg_1, reg_2
        )
        gc = get_growth_coeffs(gamma, np.ones(m) / m)
        gcs.append(gc)
    return gcs

In [85]:
data, labels = load_data_full('data/stz_4_tp_at_assignment.npz')
    
# Compute couplings
timepoints = np.unique(labels)
dfs = [data[labels == tp] for tp in timepoints]

In [5]:
for alpha in [0.01, 0.1, 1, 2, 5, 10]:
    print (alpha)
    data, labels = load_data_full('data/stz_4_tp_at_assignment.npz')
    
    # Compute couplings
    timepoints = np.unique(labels)
    dfs = [data[labels == tp] for tp in timepoints]
    
    gcs = get_all_growth_coeffs(alpha)
    gcs = np.concatenate(gcs)
    print(gcs.shape)
    np.save(f'gcs_4tp_{alpha}.npy', gcs)
    
    data, labels = load_data_full('data/stz_4_tp_at_assignment_leave_out_34.npz')
    
    # Compute couplings
    timepoints = np.unique(labels)
    dfs = [data[labels == tp] for tp in timepoints]
    
    gcs = get_all_growth_coeffs(alpha)
    gcs = np.concatenate(gcs)
    print(gcs.shape)
    np.save(f'gcs_4tp_leave_out_34_{alpha}.npy', gcs)
    
    data, labels = load_data_full('data/stz_4_tp_at_assignment_leave_out_4.npz')
    
    # Compute couplings
    timepoints = np.unique(labels)
    dfs = [data[labels == tp] for tp in timepoints]
    
    gcs = get_all_growth_coeffs(alpha)
    gcs = np.concatenate(gcs)
    print(gcs.shape)
    np.save(f'results/gcs_4tp_leave_out_4_{alpha}.npy', gcs)

0.01
(6366,)
(5600,)
(5890,)
0.1
(6366,)
(5600,)
(5890,)
1
(6366,)
(5600,)
(5890,)
2
(6366,)
(5600,)
(5890,)
5
(6366,)
(5600,)
(5890,)
10
(6366,)
(5600,)
(5890,)


## Get model

In [4]:
class GrowthNet(torch.nn.Module):
    def __init__(self):
        super().__init__()

        self.fc1 = torch.nn.Linear(11, 64)
        self.fc2 = torch.nn.Linear(64, 64)
        self.fc3 = torch.nn.Linear(64, 1)

    def forward(self, x):
        x = torch.nn.functional.leaky_relu(self.fc1(x))
        x = torch.nn.functional.leaky_relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [6]:
alpha = 2
data, labels = load_data_full('data/stz_4_tp_at_assignment.npz')
gcs = np.load(f'gcs_4tp_{alpha}.npy')
timepoints = np.unique(labels)

X = np.concatenate([data, labels[:, None]], axis=1)[labels != timepoints[-1]]
Y = gcs[:, None]

device = torch.device("cuda:" + str(1) if torch.cuda.is_available() else "cpu")

model = GrowthNet().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

for it in range(100000):
    optimizer.zero_grad()
    batch_idx = np.random.randint(len(X), size=256)
    x = torch.from_numpy(X[batch_idx,:]).type(torch.float32).to(device)
    y = torch.from_numpy(Y[batch_idx,:]).type(torch.float32).to(device)
    negative_samples = np.concatenate([np.random.uniform(size=(256,X.shape[1] - 1)) * 8 - 4,
                                       np.random.choice(timepoints, size=(256,1))], axis=1)
    negative_samples = torch.from_numpy(negative_samples).type(torch.float32).to(device)
    x = torch.cat([x, negative_samples])
    y = torch.cat([y, torch.ones_like(y)])
    pred = model(x)
    loss = torch.nn.MSELoss()
    output = loss(pred, y)
    output.backward()
    optimizer.step()
    if it % 100 == 0:
        print(it, output)

torch.save(model, 'results/stz_4_tp_at_assignment_growth_model')

0 tensor(1.3426, grad_fn=<MseLossBackward0>)
100 tensor(0.0287, grad_fn=<MseLossBackward0>)
200 tensor(0.0086, grad_fn=<MseLossBackward0>)
300 tensor(0.0087, grad_fn=<MseLossBackward0>)
400 tensor(0.0064, grad_fn=<MseLossBackward0>)
500 tensor(0.0055, grad_fn=<MseLossBackward0>)
600 tensor(0.0066, grad_fn=<MseLossBackward0>)
700 tensor(0.0049, grad_fn=<MseLossBackward0>)
800 tensor(0.0037, grad_fn=<MseLossBackward0>)
900 tensor(0.0020, grad_fn=<MseLossBackward0>)
1000 tensor(0.0032, grad_fn=<MseLossBackward0>)
1100 tensor(0.0033, grad_fn=<MseLossBackward0>)
1200 tensor(0.0021, grad_fn=<MseLossBackward0>)
1300 tensor(0.0036, grad_fn=<MseLossBackward0>)
1400 tensor(0.0050, grad_fn=<MseLossBackward0>)
1500 tensor(0.0024, grad_fn=<MseLossBackward0>)
1600 tensor(0.0027, grad_fn=<MseLossBackward0>)
1700 tensor(0.0038, grad_fn=<MseLossBackward0>)
1800 tensor(0.0018, grad_fn=<MseLossBackward0>)
1900 tensor(0.0018, grad_fn=<MseLossBackward0>)
2000 tensor(0.0027, grad_fn=<MseLossBackward0>)
2100

In [7]:
alpha = 2
data, labels = load_data_full('data/stz_4_tp_at_assignment_leave_out_4.npz')
gcs = np.load(f'gcs_4tp_leave_out_4_{alpha}.npy')
timepoints = np.unique(labels)

X = np.concatenate([data, labels[:, None]], axis=1)[labels != timepoints[-1]]
Y = gcs[:, None]

device = torch.device("cuda:" + str(1) if torch.cuda.is_available() else "cpu")

model = GrowthNet().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

for it in range(100000):
    optimizer.zero_grad()
    batch_idx = np.random.randint(len(X), size=256)
    x = torch.from_numpy(X[batch_idx,:]).type(torch.float32).to(device)
    y = torch.from_numpy(Y[batch_idx,:]).type(torch.float32).to(device)
    negative_samples = np.concatenate([np.random.uniform(size=(256,X.shape[1] - 1)) * 8 - 4,
                                       np.random.choice(timepoints, size=(256,1))], axis=1)
    negative_samples = torch.from_numpy(negative_samples).type(torch.float32).to(device)
    x = torch.cat([x, negative_samples])
    y = torch.cat([y, torch.ones_like(y)])
    pred = model(x)
    loss = torch.nn.MSELoss()
    output = loss(pred, y)
    output.backward()
    optimizer.step()
    if it % 100 == 0:
        print(it, output)

torch.save(model, 'results/stz_4_tp_at_assignment_leave_out_4_growth_model')

0 tensor(1.1659, grad_fn=<MseLossBackward0>)
100 tensor(0.0179, grad_fn=<MseLossBackward0>)
200 tensor(0.0083, grad_fn=<MseLossBackward0>)
300 tensor(0.0047, grad_fn=<MseLossBackward0>)
400 tensor(0.0030, grad_fn=<MseLossBackward0>)
500 tensor(0.0036, grad_fn=<MseLossBackward0>)
600 tensor(0.0026, grad_fn=<MseLossBackward0>)
700 tensor(0.0024, grad_fn=<MseLossBackward0>)
800 tensor(0.0034, grad_fn=<MseLossBackward0>)
900 tensor(0.0016, grad_fn=<MseLossBackward0>)
1000 tensor(0.0034, grad_fn=<MseLossBackward0>)
1100 tensor(0.0028, grad_fn=<MseLossBackward0>)
1200 tensor(0.0020, grad_fn=<MseLossBackward0>)
1300 tensor(0.0009, grad_fn=<MseLossBackward0>)
1400 tensor(0.0018, grad_fn=<MseLossBackward0>)
1500 tensor(0.0007, grad_fn=<MseLossBackward0>)
1600 tensor(0.0015, grad_fn=<MseLossBackward0>)
1700 tensor(0.0005, grad_fn=<MseLossBackward0>)
1800 tensor(0.0025, grad_fn=<MseLossBackward0>)
1900 tensor(0.0012, grad_fn=<MseLossBackward0>)
2000 tensor(0.0004, grad_fn=<MseLossBackward0>)
2100

In [8]:
alpha = 2
data, labels = load_data_full('data/stz_4_tp_at_assignment_leave_out_34.npz')
gcs = np.load(f'gcs_4tp_leave_out_34_{alpha}.npy')
timepoints = np.unique(labels)

X = np.concatenate([data, labels[:, None]], axis=1)[labels != timepoints[-1]]
Y = gcs[:, None]

device = torch.device("cuda:" + str(1) if torch.cuda.is_available() else "cpu")

model = GrowthNet().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters())

for it in range(100000):
    optimizer.zero_grad()
    batch_idx = np.random.randint(len(X), size=256)
    x = torch.from_numpy(X[batch_idx,:]).type(torch.float32).to(device)
    y = torch.from_numpy(Y[batch_idx,:]).type(torch.float32).to(device)
    negative_samples = np.concatenate([np.random.uniform(size=(256,X.shape[1] - 1)) * 8 - 4,
                                       np.random.choice(timepoints, size=(256,1))], axis=1)
    negative_samples = torch.from_numpy(negative_samples).type(torch.float32).to(device)
    x = torch.cat([x, negative_samples])
    y = torch.cat([y, torch.ones_like(y)])
    pred = model(x)
    loss = torch.nn.MSELoss()
    output = loss(pred, y)
    output.backward()
    optimizer.step()
    if it % 100 == 0:
        print(it, output)

torch.save(model, 'results/stz_4_tp_at_assignment_leave_out_34_growth_model')

0 tensor(0.8204, grad_fn=<MseLossBackward0>)
100 tensor(0.0103, grad_fn=<MseLossBackward0>)
200 tensor(0.0048, grad_fn=<MseLossBackward0>)
300 tensor(0.0045, grad_fn=<MseLossBackward0>)
400 tensor(0.0058, grad_fn=<MseLossBackward0>)
500 tensor(0.0020, grad_fn=<MseLossBackward0>)
600 tensor(0.0046, grad_fn=<MseLossBackward0>)
700 tensor(0.0039, grad_fn=<MseLossBackward0>)
800 tensor(0.0011, grad_fn=<MseLossBackward0>)
900 tensor(0.0010, grad_fn=<MseLossBackward0>)
1000 tensor(0.0010, grad_fn=<MseLossBackward0>)
1100 tensor(0.0006, grad_fn=<MseLossBackward0>)
1200 tensor(0.0005, grad_fn=<MseLossBackward0>)
1300 tensor(0.0013, grad_fn=<MseLossBackward0>)
1400 tensor(0.0004, grad_fn=<MseLossBackward0>)
1500 tensor(0.0004, grad_fn=<MseLossBackward0>)
1600 tensor(0.0003, grad_fn=<MseLossBackward0>)
1700 tensor(0.0003, grad_fn=<MseLossBackward0>)
1800 tensor(0.0002, grad_fn=<MseLossBackward0>)
1900 tensor(0.0024, grad_fn=<MseLossBackward0>)
2000 tensor(0.0002, grad_fn=<MseLossBackward0>)
2100